# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import nltk
nltk.download(['punkt','wordnet'])
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
import numpy as np


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [3]:
# load data from database
def load_data():
    engine = create_engine('sqlite:///Disaster.db')
    df = pd.read_sql_table('Disaster_Data',con=engine)
    target=[col for col in df.columns if col not in ['id','message','original','genre']]
    X = df['message']
    Y = df[target]
    
    return X,Y

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    tokens=word_tokenize(text)
    lemmatizer=WordNetLemmatizer()
    clean_tokens=[]
    
    for tok in tokens:
        clean_tok=lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
def model_pipeline():
    
    pipeline=Pipeline([
                       ('vect',CountVectorizer(tokenizer=tokenize)),
                       ('tfidf',TfidfTransformer()),
                       ('clf',RandomForestClassifier())
                      ])
    
    
    return pipeline
    

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X,y=load_data()

for col in y.columns:
    y[col]=y[col].replace({2:1})

In [7]:

X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42)

model=model_pipeline()

model.fit(X_train,y_train)

y_pred = model.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
#labels = np.unique(y_pre
cols=y.columns
for idx,col in enumerate(cols):
    print (" {} classification report is \n {}".format(col, classification_report(y_test.loc[:,col],y_pred[:,idx])))

 related classification report is 
              precision    recall  f1-score   support

          0       0.58      0.46      0.51      1563
          1       0.84      0.90      0.87      4991

avg / total       0.78      0.79      0.78      6554

 request classification report is 
              precision    recall  f1-score   support

          0       0.88      0.99      0.93      5443
          1       0.84      0.37      0.51      1111

avg / total       0.88      0.88      0.86      6554

 offer classification report is 
              precision    recall  f1-score   support

          0       0.99      1.00      1.00      6521
          1       0.00      0.00      0.00        33

avg / total       0.99      0.99      0.99      6554

 aid_related classification report is 
              precision    recall  f1-score   support

          0       0.69      0.91      0.79      3884
          1       0.77      0.41      0.54      2670

avg / total       0.72      0.71      0.69      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
pipeline=model_pipeline()
parameters = { #'vect__ngram_range':((1,1),(1,2))
               #'vect__max_df':(0.5,0.75,1.0),
               'vect__max_features':(None,5000,10000),
               #'tfidf__use_idf':(True,False)
               'clf__n_estimators':[50,100,200],
               #'clf__min_samples_split':[2,3,4] 
}

cv = GridSearchCV(pipeline,param_grid=parameters,n_jobs=-1)

cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'vect__max_features': (None, 5000, 10000), 'clf__n_estimators': [50, 100, 200]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [11]:
y_new_pred = cv.predict(X_test)

In [15]:
cols=y.columns

print(classification_report(y_test,y_pred),accuracy_score(y_test,y_pred))

             precision    recall  f1-score   support

          0       0.84      0.90      0.87      4991
          1       0.84      0.37      0.51      1111
          2       0.00      0.00      0.00        33
          3       0.77      0.41      0.54      2670
          4       0.56      0.02      0.03       535
          5       0.45      0.01      0.03       344
          6       0.00      0.00      0.00       159
          7       0.50      0.01      0.02       116
          8       0.50      0.01      0.02       200
          9       0.00      0.00      0.00         0
         10       0.88      0.13      0.22       418
         11       0.86      0.20      0.32       745
         12       0.92      0.13      0.23       581
         13       0.83      0.05      0.10        98
         14       0.86      0.05      0.09       133
         15       1.00      0.01      0.03        73
         16       0.00      0.00      0.00       215
         17       0.69      0.03      0.06   

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.